<a href="https://colab.research.google.com/github/hamzaq453/Langchain-learnings/blob/main/Chatbot_with_Neon_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot with message summarization & external DB memory

## Review

We've covered how to customize graph state schema and reducer.

We've also shown a number of tricks for trimming or filtering messages in graph state.

We've used these concepts in a Chatbot with memory that produces a running summary of the conversation.

## Goals

But, what if we want our Chatbot to have memory that persists indefinitely?

Now, we'll introduce some more advanced checkpointers that support external databases.

Here, we'll show how to use [Postgres as a checkpointer](https://langchain-ai.github.io/langgraph/how-tos/persistence_postgres/)

In [ ]:
%%capture --no-stderr
%pip install -U langgraph langgraph-checkpoint-postgres psycopg psycopg-pool langchain_google_genai


In [ ]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [ ]:
import os
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

# Use sync connection¶
This sets up a synchronous connection to the database.

Synchronous connections execute operations in a blocking manner, meaning each operation waits for completion before moving to the next one. The DB_URI is the database connection URI, with the protocol used for connecting to a PostgreSQL database, authentication, and host where database is running. The connection_kwargs dictionary defines additional parameters for the database connection.

In [ ]:
from google.colab import userdata
DB_URL = userdata.get('DB_URI')

In [ ]:
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver

# Connection pool for efficient database access
connection_kwargs = {"autocommit": True, "prepare_threshold": 0}

# Create a persistent connection pool
pool = ConnectionPool(conninfo=DB_URL, max_size=20, kwargs=connection_kwargs)

# Initialize PostgresSaver checkpointer
checkpointer = PostgresSaver(pool)
checkpointer.setup()  # Ensure database tables are set up


Let's re-define our chatbot.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage

from langgraph.graph import END
from langgraph.graph import MessagesState

model: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", api_key =  GEMINI_API_KEY)

class State(MessagesState):
    summary: str

# Define the logic to call the model
def call_model(state: State) -> State:

    # Get summary if it exists
    summary = state.get("summary", "")
    print(f"Using summary: {summary}")

    # If there is summary, then we add it
    if summary:

        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"

        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]

    else:
        messages = state["messages"]

    response = model.invoke(messages)
    return {"messages": response}

def summarize_conversation(state: State) -> State:
    print(f"Messages before summarizing: {len(state['messages'])}")
    # First, we get any existing summary
    summary = state.get("summary", "")
    print(f"Existing summary: {summary}")

    # Create our summarization prompt
    if summary:

        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )

    else:
        summary_message = "Create a summary of the conversation above:"


    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    # Summarization logic
    print(f"New summary: {response.content}")

    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]

    print(f"Messages after truncation: {len(delete_messages)}")
    return {"summary": response.content, "messages": delete_messages}

# Determine whether to end or summarize the conversation
def should_continue(state: State) -> State:

    """Return the next node to execute."""

    messages = state["messages"]
    print(f"Message count: {len(messages)}")
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"

    # Otherwise we can just end
    return END

Now, we just re-compile with our postgres checkpointer.

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph

# Redefine workflow
workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile the workflow with PostgreSQL checkpointer
graph = workflow.compile(checkpointer=checkpointer)


Now, we can invoke the graph several times.

In [ ]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="hi! I'm Hamza")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 2
================================== Ai Message ==================================

Hi Hamza!  It's nice to meet you. How can I help you today?


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Hamza", additional_kwargs={}, response_metadata={}, id='c0d209c2-b19a-4d6a-8531-220ea8970fea'), AIMessage(content="Hi Hamza!  It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-0c4d73f1-7800-4d69-af13-1dbaa0585d51-0', usage_metadata={'input_tokens': 7, 'output_tokens': 20, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}})]}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1efcf14d-fb0d-627f-8001-750da57c9919'}}, metadata={'step': 1, 'source': 'loop', 'writes': {'conversation': {'messages': AIMessage(content="Hi Hamza!  It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'safety_ratings': [], 'prompt_feedback': {'block_reason': 0, 'safety_ratings':

In [ ]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="I like Coding.")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 4
================================== Ai Message ==================================

That's great!  What kind of coding do you enjoy or are you interested in learning?  Knowing that will help me give you more specific and helpful information.  For example, are you interested in web development, game development, mobile app development, data science, or something else?


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Hamza", additional_kwargs={}, response_metadata={}, id='c0d209c2-b19a-4d6a-8531-220ea8970fea'), AIMessage(content="Hi Hamza!  It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-0c4d73f1-7800-4d69-af13-1dbaa0585d51-0', usage_metadata={'input_tokens': 7, 'output_tokens': 20, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='I like Coding.', additional_kwargs={}, response_metadata={}, id='2ff38200-4438-44b1-87ba-015360a1bb38'), AIMessage(content="That's great!  What kind of coding do you enjoy or are you interested in learning?  Knowing that will help me give you more specific and helpful information.  For example, are you interested in web development, game development, mobile app development, data science, or something else?", addi

In [ ]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="What's my name and what do i like to do ?")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 6
================================== Ai Message ==================================

Your name is Hamza, and you like coding.


StateSnapshot(values={'messages': [HumanMessage(content="hi! I'm Hamza", additional_kwargs={}, response_metadata={}, id='c0d209c2-b19a-4d6a-8531-220ea8970fea'), AIMessage(content="Hi Hamza!  It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-0c4d73f1-7800-4d69-af13-1dbaa0585d51-0', usage_metadata={'input_tokens': 7, 'output_tokens': 20, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='I like Coding.', additional_kwargs={}, response_metadata={}, id='2ff38200-4438-44b1-87ba-015360a1bb38'), AIMessage(content="That's great!  What kind of coding do you enjoy or are you interested in learning?  Knowing that will help me give you more specific and helpful information.  For example, are you interested in web development, game development, mobile app development, data science, or something else?", addi

In [ ]:
# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Start a conversation
input_message = HumanMessage(content="Can you describe about abstract paintings?")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

# Check the persisted state
graph_state = graph.get_state(config)
graph_state

Using summary: 
Message count: 8
Messages before summarizing: 8
Existing summary: 
New summary: The conversation began with an introduction where I identified myself as an AI and Hamza introduced himself.  Hamza stated his interest in coding.  I then asked him questions to better understand his coding interests.  Finally, Hamza asked me to describe abstract paintings, which I did, providing a detailed explanation of its characteristics and styles.
Messages after truncation: 6
================================== Ai Message ==================================

Abstract painting is a genre of art that doesn't attempt to represent an accurate depiction of visual reality but instead uses shapes, colours, forms, and gestural marks to achieve its effect.  It's about expressing ideas and emotions through visual language rather than literal representation.

Here are some key characteristics of abstract painting:

* **Non-representational:**  Unlike realistic or figurative art, abstract paintings 

StateSnapshot(values={'messages': [HumanMessage(content='Can you describe about abstract paintings?', additional_kwargs={}, response_metadata={}, id='5d424981-dce5-4c5e-89f5-5d6f56506afe'), AIMessage(content="Abstract painting is a genre of art that doesn't attempt to represent an accurate depiction of visual reality but instead uses shapes, colours, forms, and gestural marks to achieve its effect.  It's about expressing ideas and emotions through visual language rather than literal representation.\n\nHere are some key characteristics of abstract painting:\n\n* **Non-representational:**  Unlike realistic or figurative art, abstract paintings don't depict recognizable objects or scenes.  The focus is on the elements of art themselves – line, color, shape, texture, and composition.\n\n* **Emphasis on form and color:**  The visual elements are used to create mood, evoke feelings, or explore visual concepts.  Color choices can be highly significant, conveying emotion or creating visual ten

In [ ]:
# Retrieve state using thread ID
config = {"configurable": {"thread_id": "1"}}
graph_state = graph.get_state(config)
graph_state

StateSnapshot(values={'messages': [HumanMessage(content='Can you describe about abstract paintings?', additional_kwargs={}, response_metadata={}, id='5d424981-dce5-4c5e-89f5-5d6f56506afe'), AIMessage(content="Abstract painting is a genre of art that doesn't attempt to represent an accurate depiction of visual reality but instead uses shapes, colours, forms, and gestural marks to achieve its effect.  It's about expressing ideas and emotions through visual language rather than literal representation.\n\nHere are some key characteristics of abstract painting:\n\n* **Non-representational:**  Unlike realistic or figurative art, abstract paintings don't depict recognizable objects or scenes.  The focus is on the elements of art themselves – line, color, shape, texture, and composition.\n\n* **Emphasis on form and color:**  The visual elements are used to create mood, evoke feelings, or explore visual concepts.  Color choices can be highly significant, conveying emotion or creating visual ten

In [ ]:
pool.close()

### Persisting state

Using database like Postgres means state is persisted!

For example, we can re-start the notebook kernel and see that we can still load from Postgres DB on disk.
